Json import to openSearch

In [ ]:
import createRecipesMap as crm

crm.CreateRecipesMap("../jsonData/recipe_queries_results.json", '../jsonData/recipesMap.json').createMap()

In [ ]:
import loginOpenSearch as lop

login = lop.LoginOpenSearch("../jsonData/openSearchConfig.json").login()
    
client = login[0]
index_name = login[1]

In [ ]:
import createIndexes as ci

ci.CreateIndexes(client, index_name).createIndexStructure()

In [ ]:
import computeEmbeddings as ce

ce.ComputeEmbeddings("../jsonData/recipesMap.json","../jsonData/recipesEmbeddingsMap.json").createMap()

In [ ]:
import indexRecipes as ir

ir.IndexRecipes("../jsonData/recipesMap.json", "../jsonData/recipesEmbeddingsMap.json", client, index_name).indexRecipes()


In [ ]:
import textBasedSearch as tbs

tbs.TextBasedSearch(client, index_name).queryOpenSearch('how to make chocolate')

In [ ]:
import embeddingBasedSearch as ebs

ebs.EmbeddingBasedSearch(client, index_name).queryOpenSearch('how to make chicken')

In [ ]:
dadaddad
if client.indices.exists(index=index_name):
    # Delete the index.
    response = client.indices.delete(
        index = index_name,
        timeout = "600s"
    )
    print('\nDeleting index:')
    print(response)